In [ ]:
from sparknlp.pretrained import PretrainedPipeline

## Test Language Dectector

In [ ]:
pipeline = PretrainedPipeline("detect_language_375", lang = "xx")
pipeline.annotate("хороший")

detect_language_375 download started this may take some time.
Approx size to download 9.4 MB
[ | ][OK!]
Out[35]: {'document': ['хороший'], 'sentence': ['хороший'], 'language': ['ru']}

## Annotate Submissions

In [ ]:
import pyspark.sql.functions as F
test = spark.read.parquet("/FileStore/tables/submissions_all")
test = test.withColumn("Combine", F.concat(F.col('title'),F.lit(' '), F.col('selftext')))
results = pipeline.annotate(test, "Combine")

In [ ]:
results = results.withColumn("Russian", results.language[0].metadata.ru)
results = results.withColumn("Ukrainian", results.language[0].metadata.uk)
results = results.withColumn("English", results.language[0].metadata.en)
submission_lang = results.drop("language", "document", "sentence")

submission_lang.write.mode('overwrite').format("parquet").save("/FileStore/tables/submissions_all_lang")

## Language by Subreddit

In [ ]:
submission_sub_lang = submission_lang.groupby("subreddit", "foreign_lang").agg(F.count(F.lit(1)).alias("Counts"))
submission_sub_lang_df = submission_sub_lang.toPandas().sort_values(["subreddit"])
submission_sub_lang_df[submission_sub_lang_df["foreign_lang"] == "True"]

Out[51]:

,subreddit,foreign_lang,Counts
10,CombatFootage,True,39
17,CredibleDefense,True,31
7,RussiaUkraineWar2022,True,6972
16,UkraineConflict,True,3990
4,UkraineInvasionVideos,True,4434
9,UkraineWarReports,True,2326
23,UkraineWarVideoReport,True,13160
6,UkrainianConflict,True,21756
1,geopolitics,True,8
11,ukraina,True,8996
